In [1]:
### importing required resources
import pandas as pd
import numpy as np
from sympy import S, symbols, printing
from matplotlib import pyplot as plt
import datetime
import seaborn as sns

In [4]:
### specifying file location
file = '//north.cfs.uoguelph.ca/soe-other-home$/dhimanj/Desktop/swat_surf_temp_mod/raw.xlsx'

In [40]:
### reading the file into dataframe
df = pd.read_excel(file)

In [41]:
### getting info about dataframe size and type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118758 entries, 0 to 118757
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Location           118758 non-null  object 
 1   Date_Time          118758 non-null  object 
 2   AirTemp            118758 non-null  float64
 3   SoilTempSod_5cm    118758 non-null  float64
 4   SoilTempSod_10cm   118758 non-null  float64
 5   SoilTempSod_20cm   118758 non-null  float64
 6   TotSolarRadiation  118758 non-null  float64
 7   GrassTemp          118758 non-null  float64
dtypes: float64(6), object(2)
memory usage: 7.2+ MB


In [42]:
# dropping location column
df.drop(columns=['Location'], inplace=True)

In [43]:
# checking again
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118758 entries, 0 to 118757
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Date_Time          118758 non-null  object 
 1   AirTemp            118758 non-null  float64
 2   SoilTempSod_5cm    118758 non-null  float64
 3   SoilTempSod_10cm   118758 non-null  float64
 4   SoilTempSod_20cm   118758 non-null  float64
 5   TotSolarRadiation  118758 non-null  float64
 6   GrassTemp          118758 non-null  float64
dtypes: float64(6), object(1)
memory usage: 6.3+ MB


In [44]:
# setting datetime
df['date'] = pd.to_datetime(df['Date_Time'])

In [45]:
# removing old date column
df.drop(columns=['Date_Time'], inplace=True)

In [46]:
# dividing dataframes to two parts
# selecting df1 with al columns but one
df1 = df.loc[:, df.columns != 'TotSolarRadiation']

In [47]:
# selecting df2 for solar radiation
df2 = df[['date', 'TotSolarRadiation']]

In [48]:
# finding daily averages for temperatures
### setting date as index in the data set
df1.set_index('date')
df2.set_index('date')

### resampling data so that we have daily averaged values
df1_daily = df1.resample('D', on='date').mean()
### resampling data so that we have daily added values
df2_daily = df2.resample('D', on='date').sum()

In [49]:
# checking
df1_daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4949 entries, 2008-01-01 to 2021-07-19
Freq: D
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   AirTemp           4949 non-null   float64
 1   SoilTempSod_5cm   4949 non-null   float64
 2   SoilTempSod_10cm  4949 non-null   float64
 3   SoilTempSod_20cm  4949 non-null   float64
 4   GrassTemp         4949 non-null   float64
dtypes: float64(5)
memory usage: 232.0 KB


In [50]:
df2_daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4949 entries, 2008-01-01 to 2021-07-19
Freq: D
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   TotSolarRadiation  4949 non-null   float64
dtypes: float64(1)
memory usage: 77.3 KB


In [51]:
# resetting indeces
df1_daily.reset_index(inplace=True)
df2_daily.reset_index(inplace=True)

In [54]:
# merging on date
df3 = pd.merge(df1_daily, df2_daily, on='date')
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4949 entries, 0 to 4948
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               4949 non-null   datetime64[ns]
 1   AirTemp            4949 non-null   float64       
 2   SoilTempSod_5cm    4949 non-null   float64       
 3   SoilTempSod_10cm   4949 non-null   float64       
 4   SoilTempSod_20cm   4949 non-null   float64       
 5   GrassTemp          4949 non-null   float64       
 6   TotSolarRadiation  4949 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 309.3 KB


In [55]:
df3.head()

,date,AirTemp,SoilTempSod_5cm,SoilTempSod_10cm,SoilTempSod_20cm,GrassTemp,TotSolarRadiation
0,2008-01-01,0.010542,-1.241667,1.250625,1.198917,-0.237667,3141.129
1,2008-01-02,-5.020000,-1.235458,0.915500,1.183458,-0.847958,7858.830
2,2008-01-03,-8.861792,-1.430708,0.585750,1.111542,-3.421583,4800.850
3,2008-01-04,-3.271875,-1.701375,0.352083,0.907333,-2.440958,5230.598
4,2008-01-05,1.822292,-1.724875,0.304542,0.812250,-0.561875,1730.801


In [56]:
# writing to excel
df3.to_excel(r'//north.cfs.uoguelph.ca/soe-other-home$/dhimanj/Desktop/swat_surf_temp_mod/daily.xlsx')